### git setup

#### start version control of all the files and put the files inside the drive , bcoz temporary colab session can delete this file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir /content/drive/MyDrive/Github/IMG2TEXTe


In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/IMG2TEXTe.ipynb" /content/drive/MyDrive/Github/IMG2TEXTe/


In [ ]:
%cd drive/MyDrive/Github/IMG2TEXTe/

In [ ]:
!git init

### Requirements.txt file
Google colab has most of the dependencies already installed(for instance, we dont need to install pandas , tensorflow or a pyhton ) so this req.txt file only contains dependencies which have to be explicitly installed

In [ ]:
%%writefile requirements.txt
pytesseract==0.3.13
streamlit==1.38.0
pillow_heif==0.18.0
pyngrok==7.2.0



#### installing system level software --> tesseract-ocr  with sudo apt -get

In [ ]:
!pip install -r requirements.txt -q

In [ ]:
!sudo apt-get install tesseract-ocr -q

#### checking installation

In [ ]:
import pytesseract
print(pytesseract.get_tesseract_version())

In [ ]:
#addding tesseract to the path
pytesseract.pytesseract.tesseract_cmd = "/bin/tesseract"

In [ ]:
!which tesseract

### Image_processing File

In [ ]:
%%writefile image_processing.py
import cv2
import pytesseract
# import asyncio
def getTextFromImage(img):
  grayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  ret, thresh1 = cv2.threshold(grayImg, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
  rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25,25))
  contours, heirarchy = cv2.findContours(thresh1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE )
  grayImg2 = grayImg.copy()
  cnt_list = [] #stores output text and its position co-ordinate(x,y)
  for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)

  #Drawing the fixed definite Rectangle Boundary around the irregular shaped contour
  rect = cv2.rectangle(grayImg2,(x,y), (x+w, y+h), (0,255,0),2)
  #Drawing the circle to mark the start of the text region
  cv2.circle(grayImg2,(x,y), 8, (255,255,0), 8)
  #do 2D Numpy slicing to only get the detected text region, that means to slice out the rectangle boundary by using its boundary co-ordinates
  croppedImgRegions = grayImg2[y:y+h, x:x+w]
  #Now apply OCR on the cropped image
  text = pytesseract.image_to_string(croppedImgRegions)
  cnt_list.append([x,y, text])
  #this function sorts the text block from top to bottom(stick with y for now, x later if needed)
  sorted_list = sorted(cnt_list, key = lambda x: x[1])
  file = open("scanned_textfile", "w+")
  file.write("")
  file.close()

  for(x,y,text) in sorted_list:
    file = open("scanned_textfile", "w")
    #appending the text
    file.write(text)
    file.write("\n")
    file.close()
  with open("scanned_textfile", "r") as f:
    textContent = f.read()
    return textContent



### Streamlit app

In [ ]:
%%writefile app.py
# magic command from jupyter notebook, writes whatever content insdie the cell to the .py file
import streamlit as st
import cv2
import numpy as np
from PIL import Image
import io #pyhton built-in module to deal with input/output Memory Streams
import time
from image_processing import getTextFromImage
html_temp = """
            <div style="background-color:{};padding:1px">

            </div>
            """

textContent = None
#SIDEBAR

with st.sidebar:
  st.title(":smile: Text from Image")
  st.markdown(html_temp.format("rgba(55, 53, 47, 0.16)"),unsafe_allow_html=True)
  st.markdown("""
    # How does it work?
    Simply upload your images and get the extracted the Text content.
    """)
  st.markdown(html_temp.format("rgba(55, 53, 47, 0.16)"),unsafe_allow_html=True)
# #CAMERA UPLOAD --LATER
# img_file_buffer = st.camera_input("Take a picture")
# #opening Image file for Image Processing tasks via OpenCV
# if img_file_buffer is not None:
#   bytes_data = img_file_buffer.getvalue()
#   cv2_img = cv2.imdecode(np.frombuffer(bytes_data, np.uint8),cv2.IMREAD_COLOR)
#   #check image type
#   st.write(type(cv2_img))

# FILE_UPLOAD
st.title("📷 Image Upload and Processing")
img_file = st.file_uploader("Upload Image file", accept_multiple_files=False)
print(img_file)

cv2_img = None
if img_file is not None:
    file_details = {"fileName": img_file.name, "FileType": img_file.type, "FileSize": f"{img_file.size/1024:.2f} KB"}
    try:
        if img_file.type == "image/heic":
            from pillow_heif import register_heif_opener
            register_heif_opener()
            image = Image.open(img_file)
            image = image.convert('RGB')
            buf = io.BytesIO()
            image.save(buf, format='JPEG')
            jpeg_bytes = buf.getvalue()
        else:
            jpeg_bytes = img_file.read()

        cv2_img = cv2.imdecode(np.frombuffer(jpeg_bytes, np.uint8), cv2.IMREAD_COLOR) #cv2_img is in np array format
        # st.write(type(cv2_img))
        # st.image(cv2_img, channels="BGR", caption="Uploaded Image")
        # st.write("Image shape: ", cv2_img.shape)
    except Exception as e:
        st.error(f"An error occurred: {str(e)}")
        st.warning("Please upload an image file")



st.markdown("""
<style>
.big-gap {
    margin-bottom: 2em;
}
</style>
""", unsafe_allow_html=True)

st.markdown('<p class="big-gap"></p>', unsafe_allow_html=True)


#OUTPUTS::
# Custom CSS for the blinking effect
st.markdown("""
<style>
@keyframes blink {
    0% { opacity: 0; }
    50% { opacity: 1; }
    100% { opacity: 0; }
}
.blinking {
    animation: blink 1.5s linear infinite;
}
</style>
""", unsafe_allow_html=True)

# Function to create a blinking loading message
def blinking_loading_message(message):
    return f'<p class="blinking">{message}</p>'

if cv2_img is not None:
    col1, col2 = st.columns(2, gap="large")
    with col1:
        st.subheader("Image Column")
        image_placeholder = st.empty()
        loading_message = st.empty()
        loading_message.text("Loading image...")
        time.sleep(1)  # Reduced sleep time
        image_placeholder.image(cv2_img)
        loading_message.empty()

    try:
        textContent = getTextFromImage(cv2_img)
    except Exception as e:
        st.error(f"An error occurred while extracting text: {str(e)}")
else:
    st.info("Please upload an image to see the content")

# Column 2: Text content with blinking loading sign
if textContent is not None:
  with col2:
    st.subheader("Text Content Column")

    # Placeholder for text content
    text_placeholder = st.empty()

    # Blinking loading message
    loading_message = st.markdown(blinking_loading_message("Loading text content..."), unsafe_allow_html=True)

    # # Simulate text content loading
    # time.sleep(3)  # Simulate longer delay for text content

    text_placeholder.text(textContent)
    st.write(len(textContent))
    loading_message.empty()
else:
  with col2:
    st.write("No text")






### Ngrok tunneling into Streamlit
Start pyngrok tunneling to access streamlit inside local network of google colab server

Step 1: Go to the ngrok and get the authorization token and store it as google colab secret key

In [ ]:
from pyngrok import ngrok

In [ ]:
from google.colab import userdata
NGROK_T = userdata.get('NGROKT')
# print(NGROK_T)

In [ ]:
!ngrok authtoken $NGROK_T

In [ ]:
!nohup streamlit run app.py --server.port 8502 &

In [ ]:
 #Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='8502', proto='http', bind_tls=True)

In [ ]:
tunnels = ngrok.get_tunnels()
print(tunnels)


In [ ]:
#Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)